**Nhận dạng số MNIST sử dụng máy học**

Sử dụng Google Colab để nhận dạng số MNIST. Việc gắn kết Google Drive vào Colab có một số mục đích, bao gồm:
1.   Truy nhập dữ liệu
2.   Lưu trữ và chia sẻ kết quả
3.   Sử dụng dữ liệu trong dự Machine Learning và Deep Learning (Có thể sử dụng Google Drive để lưu trữ dữ liệu đào tạo, mô hình và các tệp cần thiết khác)
4.   Sử dụng lưu trữ miễn phí (Google Drive cung cấp 15 GB lưu trữ miễn phí)

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


Đầu tiên, ta cần import các thư viện cần thiết như:
*   **tensorflow.compat.v1**: Dùng để xây dựng và huấn luyện mô hình máy học. Chúng ta chỉ định trực tiếp đây là phiên bản TensorFlow 1.x
*   **matplotlib.pyplot**: Trực quan hóa dữ liệu thông qua biểu đồ và đồ thị
*   **random**: Tạo số ngẫu nhiên



In [2]:
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
import random

In [9]:
# Tắt Eager Execution (Eager mode)
tf.disable_eager_execution()

In [10]:
# Đặt giống ngẫu nhiên
tf.set_random_seed(777)

In [11]:
# Tải dữ liệu MNIST từ TensorFlow Datasets
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [12]:
# Chuẩn hóa dữ liệu và chuyển đổi thành one-hot encoding
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train = tf.one_hot(y_train, 10)
y_test = tf.one_hot(y_test, 10)

In [13]:
# MNIST data image of shape 28 * 28 = 784
X = tf.placeholder(tf.float32, [None, 784])
# 0-9 digits recognition = 10 classes
Y = tf.placeholder(tf.float32, [None, 10])

In [14]:
W = tf.Variable(tf.random.normal([784, 10]))
b = tf.Variable(tf.random.normal([10]))

In [15]:
# Hypothesis (using softmax)
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.math.log(hypothesis), axis=1))
train = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [16]:
# Test model
is_correct = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))

In [17]:
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [ ]:
# Create a TensorFlow session
with tf.compat.v1.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.compat.v1.global_variables_initializer())

    # Training cycle
    num_epochs = 15
    batch_size = 100
    num_iterations = x_train.shape[0] // batch_size

    for epoch in range(num_epochs):
        avg_cost = 0
        for i in range(num_iterations):
            batch_xs = x_train[i * batch_size : (i + 1) * batch_size].reshape(-1, 784)
            batch_ys = y_train[i * batch_size : (i + 1) * batch_size].eval(session=tf.compat.v1.Session())
            _, cost_val = sess.run([train, cost], feed_dict={X: batch_xs, Y: batch_ys})
            avg_cost += cost_val / num_iterations
        print("Epoch: {:04d}, Cost: {:.9f}".format(epoch + 1, avg_cost))

    print("Learning finished")

    # Test the model using test sets
    test_accuracy = accuracy.eval(session=sess, feed_dict={X: x_test.reshape(-1, 784), Y: y_test.eval(session=tf.compat.v1.Session())})
    print("Accuracy: ", test_accuracy)

    # Get one and predict
    r = random.randint(0, x_test.shape[0] - 1)

    print("Label: ", sess.run(tf.argmax(y_test[r], 0)))
    print("Prediction: ", sess.run(tf.argmax(hypothesis, 1), feed_dict={X: x_test[r].reshape(-1, 784)}))
    plt.imshow(x_test[r], cmap="gray")
    plt.show()

Epoch: 0001, Cost: 2.830966463
Epoch: 0002, Cost: 1.076558116
Epoch: 0003, Cost: 0.848823462
Epoch: 0004, Cost: 0.738022550
Epoch: 0005, Cost: 0.669214803
Epoch: 0006, Cost: 0.621005244
